oft wird rag gebaut mit pdfs, und wenn man so ein system aufbaut kann man sicherlich eine grössere user base ansprechen, welche das system dann auch bauen möchten. in diesem projekt möchte ich aber webscraping von yahoo finance machen, weil viele leute auch yahoo finance lesen und auf yahoo finance viel mehr informationen sind. ich kann dort webscraping machen von bestimmten aktien, fonds oder indexes und dann alle informationen scrapen von kursen, artikeln, stock prices,
intraday trading data,
option prices;
company disclosures and information such as :
financial information,
earnings,
analyst ratings and recommendations,
insider trades,
SEC filings,
news,
large shareholders;

 dann kann ich all das in einen ordner speichern und einen rag bauen auf all diese sachen. das spart einerseits zeit weil man nicht manuell nach pdfs online suchen muss zu den gewünschten themen, sondern man kann alles zeitsparend an einem ort scrapen und dann weiterbenutzen als input für das rag.

 denn technisch gesehen macht LangChain keinen Unterschied, ob deine Daten ursprünglich aus PDFs, TXTs, Webseiten oder JSON stammen. Wichtig ist nur, was in den vektorstore eingespeist wird, ist reiner text, nicht das ursprüngliche Dateiformat (wie z.b. pdf).

 Ob du folgenden Text aus einem PDF extrahierst:
 In 2023, Allianz reported a revenue of €152.7 billion and a net income of €14.2 billion.

oder denselben Text aus einem JSON-Feld zusammenstellst wie:
{ "revenue": "€152.7 billion", "net_income": "€14.2 billion" }

ist für den Vektorstore und LangChain völlig egal, solange du am Ende diesen Text als Chunk übergibst.

📘 Warum sieht man online meist PDFs?

PDFs sind realistische Quellen (z. B. Geschäftsberichte, Studien)
Sie demonstrieren komplexere Verarbeitung (PDF → Text → Chunk → Embedding)
Viele Nutzer haben genau solche Dateien → Tutorials wollen für die breite Masse relevant sein
JSON ist oft maschinenlesbar, aber in der Praxis weniger "typisch" für Endnutzer
✅ Wann ist JSON sogar besser?

Wenn du strukturierte, tabellarische Daten hast (Finanzdaten, ESG-KPIs, Zeitreihen)
Wenn du Daten aus APIs sammelst (z. B. von Morningstar, AlphaVantage, Unternehmensdatenbanken)
Wenn du dein System voll automatisiert aufbaust ohne manuelles PDF-Verarbeiten
Wenn du bestimmte Felder gezielt extrahieren und in Textform bringen willst

Ja, du kannst problemlos ein LangChain-basiertes RAG-System mit JSON-Dateien aufbauen.
🧱 Die einzige Voraussetzung: Du musst den Inhalt in Klartext-Chunks bringen, damit das LLM damit arbeiten kann.



# source .venv/bin/activate 

In [ ]:
import yfinance as yf
import pandas
import matplotlib
from datetime import datetime
from pytz import timezone


In [ ]:
# stock price download
# by default hat man nur daily prices
# all prices are adjusted for dividends and splits

yf.download(['AAPL', 'NVDA'])

[*********************100%***********************]  2 of 2 completed


Price            Close                    High                     Low  \
Ticker            AAPL        NVDA        AAPL        NVDA        AAPL   
Date                                                                     
1980-12-12    0.098726         NaN    0.099155         NaN    0.098726   
1980-12-15    0.093575         NaN    0.094005         NaN    0.093575   
1980-12-16    0.086707         NaN    0.087136         NaN    0.086707   
1980-12-17    0.088853         NaN    0.089282         NaN    0.088853   
1980-12-18    0.091429         NaN    0.091859         NaN    0.091429   
...                ...         ...         ...         ...         ...   
2025-04-08  172.419998   96.300003  190.339996  105.849998  169.210007   
2025-04-09  198.850006  114.330002  200.610001  115.099998  171.889999   
2025-04-10  190.419998  107.570000  194.779999  110.860001  183.000000   
2025-04-11  198.149994  110.930000  199.539993  111.550003  186.059998   
2025-04-14  202.369995  110.370003  212.940002  114.290001  201.162094   

Price                         Open                 Volume               
Ticker            NVDA        AAPL        NVDA       AAPL         NVDA  
Date                                                                    
1980-12-12         NaN    0.098726         NaN  469033600          NaN  
1980-12-15         NaN    0.094005         NaN  175884800          NaN  
1980-12-16         NaN    0.087136         NaN  105728000          NaN  
1980-12-17         NaN    0.088853         NaN   86441600          NaN  
1980-12-18         NaN    0.091429         NaN   73449600          NaN  
...                ...         ...         ...        ...          ...  
2025-04-08   94.459999  186.699997  103.809998  120859500  476243400.0  
2025-04-09   97.529999  171.949997   98.889999  184395900  612918300.0  
2025-04-10   99.150002  189.070007  109.370003  121880000  437812400.0  
2025-04-11  107.480003  186.100006  108.500000   87300000  312853800.0  
2025-04-14  109.070000  211.440002  114.139999   65385598  167767788.0  

[11175 rows x 10 columns]

In [ ]:
# data for a specific stock 
# dictionary of meta data and information about a specific stock

aapl = yf.Ticker('AAPL')

aapl.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [18]:
aapl.news

[{'id': '346e3739-8006-4bf4-be1d-4997396dc773',
  'content': {'id': '346e3739-8006-4bf4-be1d-4997396dc773',
   'contentType': 'STORY',
   'title': "'Trump does Tim Apple a solid': Investors cheer smartphone tariff exemption despite mixed White House signals",
   'description': '',
   'summary': 'President Trump stoked confusion this past weekend about tariffs on smartphones, but investors are viewing an electronics exemption announced Friday night as an important temporary win for Apple and other China-dependent technology giants.',
   'pubDate': '2025-04-14T12:42:35Z',
   'displayTime': '2025-04-14T16:40:36Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-02/996ee540-f24d-11ef-bf5f-be1985ce6685',
    'originalWidth': 5118,
    'originalHeight': 3414,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/ZlGdQJTJlY8KjzjxQ4tEvA--~B/aD0zNDE0O3c9NTExODthcHB